In [1]:
import pandas as pd
import numpy as np

In [2]:
crime_data = pd.read_csv('../new_data/zip_crime_data.csv')

In [3]:
raw_restaurant_data = pd.read_csv('../new_data/restaurant_data.csv')
restaurant_zip_sum_data = raw_restaurant_data.groupby(['zipcode']).sum().rename(index=int, columns={'count':'res_total_count'}).reset_index()
restaurant_zip_max_data = raw_restaurant_data.groupby(['zipcode']).max().rename(index=int, columns={'count':'res_max_type_count', 'type':'res_max_type'}).reset_index()
restaurant_data = restaurant_zip_max_data.join(restaurant_zip_sum_data.set_index('zipcode'), on='zipcode')

In [4]:
ny_house_sell_data = pd.read_csv('../new_data/ny_house_sell.csv')
ny_house_sell_data['avg_house_sell_price'] = ny_house_sell_data.drop('zipcode', axis='columns').mean(axis=1)
ny_house_sell_data = ny_house_sell_data[['zipcode', 'avg_house_sell_price']]
ny_house_rent_data = pd.read_csv('../new_data/ny_house_rent.csv')
ny_house_rent_data['avg_house_rent_price'] = ny_house_rent_data.drop('zipcode', axis='columns').mean(axis=1)
ny_house_rent_data = ny_house_rent_data[['zipcode', 'avg_house_rent_price']]

In [5]:
lib_data = pd.read_csv('../data/NYC_Lib_zip_count.csv')\
                .rename(index=int, columns={"count1": "lib_count1", "count5": "lib_count5", "count15":"lib_count15"})

In [6]:
mus_data = pd.read_csv('../data/NYC_Mus_zip_count.csv')\
                .rename(index=int, columns={"count1": "mus_count1", "count5": "mus_count5", "count15":"mus_count15"})

In [7]:
def my_stupid_approach(store_data):
    new_data = store_data.groupby('Address ZIP').sum().reset_index()
    new_data['store_business_type_count'] = new_data['store_individual_type_count'] = 0
    for index, row in store_data.iterrows():
        if row['store_type'] == 'Business':
            new_data.loc[new_data['Address ZIP'] == row['Address ZIP'], 'store_business_type_count'] = row['store_count']
        else:
            new_data.loc[new_data['Address ZIP'] == row['Address ZIP'], 'store_individual_type_count'] = row['store_count']
    return new_data

store_data = pd.read_csv('../data/NYC_Store_Count.csv')\
        .rename(index=int, columns={"License Type": "store_type", "DCA License Number":"store_count"})
store_data = my_stupid_approach(store_data)[['Address ZIP', 'store_business_type_count', 'store_individual_type_count']]

In [8]:
univ_data = pd.read_csv('../data/NYC_Univ_zip_count.csv')\
        .rename(index=int, columns={"count1": "univ_count1", "count5": "univ_count5", "count15":"univ_count15"})

In [9]:
popu_data = pd.read_csv('../data/popu_zip.csv')[['zipcode', 'population']]

In [10]:
hospital_data = pd.read_csv('../data/NYC_hospital_zip_count.csv')\
                .rename(index=int, columns={"count10": "hospital_count10", "count5": "hospital_count5", "count15":"hospital_count15"})

In [11]:
school_data = pd.read_csv('../data/NYC_school_zip_count.csv')\
                    .rename(index=int, columns={"count":"school_count"}).groupby('Address ZIP')['school_count'].sum().reset_index()

In [12]:
park_data = pd.read_csv('../data/NYU_parkArea_zip.csv')\
                    .rename(index=str, columns={"count":"park_area"})

In [13]:
dataset = crime_data
dataset = dataset.join(restaurant_data.set_index('zipcode'), on='zipcode')
dataset = dataset.join(ny_house_sell_data.set_index('zipcode'), on='zipcode')
dataset = dataset.join(ny_house_rent_data.set_index('zipcode'), on='zipcode')
dataset = dataset.join(lib_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(mus_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(store_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(univ_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(popu_data.set_index('zipcode'), on='zipcode')
dataset = dataset.join(hospital_data.set_index('Address ZIP'), on='zipcode')
# dataset = dataset.join(school_data.set_index('Address ZIP'), on='zipcode')
dataset = dataset.join(park_data.set_index('Address ZIP'), on='zipcode')

In [14]:
dataset.head(1).transpose()

,0
date,09/30/2017
time,23:46:00
lat,40.8011
lon,-73.9505
LEVEL_FELONY,1
LEVEL_MISDEMEANOR,0
LEVEL_VIOLATION,0
zipcode,10026
res_max_type,Vegetarian
res_max_type_count,194


In [15]:
dataset = dataset.fillna({'res_max_type':'Unknown',
                          'res_max_type_count': 0, 
                          'res_total_count':0, 
                          'avg_house_sell_price':dataset['avg_house_sell_price'].mean(), 
                          'avg_house_rent_price':dataset['avg_house_rent_price'].mean(), 
                          'lib_count1':0, 
                          'lib_count5':0, 
                          'lib_count15':0, 
                          'mus_count1':0, 
                          'mus_count5':0, 
                          'mus_count15':0, 
                          'store_business_type_count':0, 
                          'store_individual_type_count':0, 
                          'univ_count1':0, 
                          'univ_count5':0, 
                          'univ_count15':0, 
                          'population':dataset['population'].mean(), 
                          'hospital_count5':0, 
                          'hospital_count10':0, 
                          'hospital_count15':0, 
                          'school_count':0,
                          'park_area':0})

In [16]:
dataset

,date,time,lat,lon,LEVEL_FELONY,LEVEL_MISDEMEANOR,LEVEL_VIOLATION,zipcode,res_max_type,res_max_type_count,...,store_business_type_count,store_individual_type_count,univ_count1,univ_count5,univ_count15,population,hospital_count5,hospital_count10,hospital_count15,park_area
0,09/30/2017,23:46:00,40.801064,-73.950482,1,0,0,10026,Vegetarian,194.0,...,118.0,134.0,0.0,17.0,124.0,34003.00000,14.0,64.0,130.0,569.897094
1,09/30/2017,23:20:00,40.677407,-74.006397,0,1,0,11231,Vietnamese/Cambodian/Malaysia,456.0,...,202.0,89.0,0.0,1.0,11.0,33336.00000,0.0,0.0,0.0,1410.767771
2,09/30/2017,23:20:00,40.623227,-74.149227,0,1,0,10302,Spanish,98.0,...,166.0,88.0,0.0,3.0,23.0,19088.00000,7.0,20.0,35.0,527.570103
3,09/30/2017,23:20:00,40.654698,-73.907624,0,0,1,11212,Spanish,207.0,...,321.0,96.0,0.0,0.0,0.0,84500.00000,14.0,40.0,82.0,1496.452799
4,09/30/2017,23:15:00,40.747495,-73.885321,1,0,0,11372,Thai,722.0,...,523.0,95.0,0.0,2.0,13.0,66636.00000,15.0,39.0,64.0,263.435729
5,09/30/2017,23:10:00,40.820204,-73.863637,0,1,0,10473,Spanish,60.0,...,113.0,118.0,0.0,0.0,0.0,58519.00000,0.0,0.0,0.0,575.000734
6,09/30/2017,23:10:00,40.770794,-73.933146,1,0,0,11102,Spanish,144.0,...,203.0,105.0,0.0,0.0,8.0,34133.00000,0.0,6.0,23.0,1504.581831
7,09/30/2017,23:09:00,40.694554,-73.802113,0,1,0,11451,American,18.0,...,0.0,0.0,0.0,6.0,40.0,0.00000,6.0,24.0,44.0,347.542281
8,09/30/2017,23:05:00,40.677098,-73.825040,1,0,0,11419,Vegetarian,285.0,...,413.0,252.0,0.0,0.0,10.0,47211.00000,0.0,0.0,6.0,134.107500
9,09/30/2017,23:02:00,40.756645,-73.986285,0,1,0,10018,Vietnamese/Cambodian/Malaysia,932.0,...,321.0,31.0,0.0,7.0,55.0,5229.00000,0.0,0.0,0.0,109.251121


In [17]:
dataset.to_csv('../data/all_data.csv', sep=',', index=False)